In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import tensorflow as tf
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Input, SimpleRNN, Dense, LSTM, GRU, Dropout
from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils
from sklearn.metrics import mean_squared_error
%matplotlib inline

In [2]:
f = open('Nasa_LiB_B5_discharge.json')
data = json.load(f)

In [3]:
f2 = open('B0005_discharge.json')
data2 = json.load(f2)

In [4]:
dataset = pd.read_json('Nasa_LiB_B5_discharge.json').T
dataset

,amb_temp,date_time,voltage_battery,current_battery,temp_battery,current_load,voltage_load,time,Capacity
1,24,"02 Apr 2008, 15:25:41","[4.191491807505295, 4.190749067776103, 3.97487...","[-0.004901589207462, -0.0014780055516420002, -...","[24.330033885570543, 24.325993424022467, 24.38...","[-0.0006000000000000001, -0.000600000000000000...","[0.0, 4.206, 3.062, 3.03, 3.011, 2.991, 2.977,...","[0.0, 16.781, 35.702999999999996, 53.781, 71.9...",[1.8564874208181572]
3,24,"02 Apr 2008, 19:43:48","[4.189773213846608, 4.188881380279099, 3.97915...","[2.1251179810807652e-05, -0.00087947126634, -2...","[24.697751935729325, 24.685948163596, 24.73813...","[-0.0006000000000000001, -0.000600000000000000...","[0.0, 4.204, 3.056, 3.024, 3.003, 2.983, 2.973...","[0.0, 16.672, 35.703, 53.828, 71.891, 90.047, ...",[1.846327249719927]
5,24,"03 Apr 2008, 00:01:06","[4.188186735991303, 4.187198338393574, 3.97999...","[-0.0017540301662320001, -0.001791647748693000...","[24.734266163954402, 24.74050014862512, 24.791...","[-0.0006000000000000001, -0.000600000000000000...","[0.0, 4.203, 3.055, 3.023, 3.0, 2.981, 2.96999...","[0.0, 16.796999999999997, 35.766, 53.891, 72.0...",[1.835349194223407]
7,24,"03 Apr 2008, 04:16:37","[4.188461118855572, 4.187752210174129, 3.98161...","[-0.002775036129446, 0.00023371270769300003, -...","[24.65423646922845, 24.652243741292516, 24.697...","[-0.0006000000000000001, -0.000600000000000000...","[0.0, 4.203, 3.055, 3.023, 2.999, 2.982, 2.969...","[0.0, 16.766000000000002, 35.75, 53.813, 72.04...",[1.835262527582112]
9,24,"03 Apr 2008, 08:33:25","[4.188298524761055, 4.187384730878918, 3.98225...","[-0.007980866803888001, 0.0017014548245030002,...","[24.524796959348127, 24.518700434435164, 24.57...","[-0.0006000000000000001, -0.000600000000000000...","[0.0, 4.203, 3.056, 3.025, 3.0, 2.984, 2.973, ...","[0.0, 16.719, 35.656000000000006, 53.859, 72.0...",[1.834645508212041]
...,...,...,...,...,...,...,...,...,...
599,24,"26 May 2008, 10:44:38","[4.196640912532072, 4.195891409328115, 3.97041...","[-0.0031723450500130003, -0.001165548113920000...","[23.741424141669572, 23.73611267134457, 23.762...","[0.0006000000000000001, 0.0006000000000000001,...","[0.0, 4.211, 2.944, 2.931, 2.92, 2.906, 2.894,...","[0.0, 9.343, 19.483999999999998, 28.7650000000...",[1.293463613844243]
603,24,"26 May 2008, 15:30:43","[4.195623574912322, 4.195225391012888, 3.96944...","[-0.005227006197317001, -0.0018109194843490002...","[23.80397851847974, 23.80026564355092, 23.8387...","[0.0006000000000000001, 0.0006000000000000001,...","[0.0, 4.211, 2.926, 2.93, 2.916, 2.902, 2.8890...","[0.0, 9.359000000000002, 19.562000000000005, 2...",[1.288003392619118]
607,24,"26 May 2008, 20:21:04","[4.195862008850563, 4.195180368506587, 3.96948...","[-0.0005545264293170001, 0.001394789927994, -2...","[23.849121770252687, 23.85032548167981, 23.872...","[0.0006000000000000001, 0.0006000000000000001,...","[0.0, 4.211, 2.939, 2.9290000000000003, 2.916,...","[0.0, 9.313000000000002, 19.469, 28.797, 38.14...",[1.28745252213794]
611,24,"27 May 2008, 15:52:41","[4.18482689434833, 4.184358228897457, 3.964730...","[-0.0038126960802460003, -0.003553288376752, -...","[24.93410482186201, 24.92195746617458, 24.9390...","[0.0006000000000000001, 0.0006000000000000001,...","[0.0, 4.2, 2.977, 2.947, 2.931, 2.917, 2.90500...","[0.0, 9.375, 19.593999999999998, 28.9839999999...",[1.309015364230735]


In [5]:
dataset2 = pd.read_json('B0005_discharge.json').T
dataset2

,amb_temp,date_time,voltage_battery,current_battery,temp_battery,current_load,voltage_load,time
1,24,"02 Apr 2008, 15:25:41","[4.191491807505295, 4.190749067776103, 3.97487...","[-0.004901589207462, -0.0014780055516420002, -...","[24.330033885570543, 24.325993424022467, 24.38...","[-0.0006000000000000001, -0.000600000000000000...","[0.0, 4.206, 3.062, 3.03, 3.011, 2.991, 2.977,...","[0.0, 16.781, 35.702999999999996, 53.781, 71.9..."
3,24,"02 Apr 2008, 19:43:48","[4.189773213846608, 4.188881380279099, 3.97915...","[2.1251179810807652e-05, -0.00087947126634, -2...","[24.697751935729325, 24.685948163596, 24.73813...","[-0.0006000000000000001, -0.000600000000000000...","[0.0, 4.204, 3.056, 3.024, 3.003, 2.983, 2.973...","[0.0, 16.672, 35.703, 53.828, 71.891, 90.047, ..."
5,24,"03 Apr 2008, 00:01:06","[4.188186735991303, 4.187198338393574, 3.97999...","[-0.0017540301662320001, -0.001791647748693000...","[24.734266163954402, 24.74050014862512, 24.791...","[-0.0006000000000000001, -0.000600000000000000...","[0.0, 4.203, 3.055, 3.023, 3.0, 2.981, 2.96999...","[0.0, 16.796999999999997, 35.766, 53.891, 72.0..."
7,24,"03 Apr 2008, 04:16:37","[4.188461118855572, 4.187752210174129, 3.98161...","[-0.002775036129446, 0.00023371270769300003, -...","[24.65423646922845, 24.652243741292516, 24.697...","[-0.0006000000000000001, -0.000600000000000000...","[0.0, 4.203, 3.055, 3.023, 2.999, 2.982, 2.969...","[0.0, 16.766000000000002, 35.75, 53.813, 72.04..."
9,24,"03 Apr 2008, 08:33:25","[4.188298524761055, 4.187384730878918, 3.98225...","[-0.007980866803888001, 0.0017014548245030002,...","[24.524796959348127, 24.518700434435164, 24.57...","[-0.0006000000000000001, -0.000600000000000000...","[0.0, 4.203, 3.056, 3.025, 3.0, 2.984, 2.973, ...","[0.0, 16.719, 35.656000000000006, 53.859, 72.0..."
...,...,...,...,...,...,...,...,...
599,24,"26 May 2008, 10:44:38","[4.196640912532072, 4.195891409328115, 3.97041...","[-0.0031723450500130003, -0.001165548113920000...","[23.741424141669572, 23.73611267134457, 23.762...","[0.0006000000000000001, 0.0006000000000000001,...","[0.0, 4.211, 2.944, 2.931, 2.92, 2.906, 2.894,...","[0.0, 9.343, 19.483999999999998, 28.7650000000..."
603,24,"26 May 2008, 15:30:43","[4.195623574912322, 4.195225391012888, 3.96944...","[-0.005227006197317001, -0.0018109194843490002...","[23.80397851847974, 23.80026564355092, 23.8387...","[0.0006000000000000001, 0.0006000000000000001,...","[0.0, 4.211, 2.926, 2.93, 2.916, 2.902, 2.8890...","[0.0, 9.359000000000002, 19.562000000000005, 2..."
607,24,"26 May 2008, 20:21:04","[4.195862008850563, 4.195180368506587, 3.96948...","[-0.0005545264293170001, 0.001394789927994, -2...","[23.849121770252687, 23.85032548167981, 23.872...","[0.0006000000000000001, 0.0006000000000000001,...","[0.0, 4.211, 2.939, 2.9290000000000003, 2.916,...","[0.0, 9.313000000000002, 19.469, 28.797, 38.14..."
611,24,"27 May 2008, 15:52:41","[4.18482689434833, 4.184358228897457, 3.964730...","[-0.0038126960802460003, -0.003553288376752, -...","[24.93410482186201, 24.92195746617458, 24.9390...","[0.0006000000000000001, 0.0006000000000000001,...","[0.0, 4.2, 2.977, 2.947, 2.931, 2.917, 2.90500...","[0.0, 9.375, 19.593999999999998, 28.9839999999..."


In [6]:
X=None
Y=None

for x in data2:
    arr=np.concatenate((np.full(99, data2[x]['voltage_battery'][0]),np.array(data2[x]['voltage_battery'])))
    for  i in range(0,len(arr)-101):
        if X is None :
          X=arr[i:i+100][np.newaxis]
          Y=arr[i+1:i+101][np.newaxis]
        else:
          X=np.concatenate((X,arr[i:i+100][np.newaxis]),axis=0)
          Y=np.concatenate((Y,arr[i+1:i+101][np.newaxis]),axis=0)
          #print(X.shape)

In [7]:
ord=np.arange(X.shape[0])
np.random.shuffle(ord)
X=X[ord]
Y=Y[ord]
X_train,Y_train=X[0:35000],Y[0:35000]
X_validate,Y_validate=X[35000:43000],Y[35000:43000]
X_test,Y_test=X[43000:],Y[43000:]

X_train = X_train.reshape(-1, 100,1)
X_validate  = X_validate.reshape(-1, 100,1)
X_test = X_test.reshape(-1, 100, 1)

Y_train = Y_train.reshape(-1, 100,1)
Y_validate  = Y_validate.reshape(-1, 100,1)
Y_test = Y_test.reshape(-1, 100, 1)

In [8]:
model = Sequential()
model.add(Dense(8, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(1))
model.summary()
model.compile(optimizer=Adam(beta_1=0.9, beta_2=0.999, epsilon=1e-08), loss='mean_absolute_error')
model.fit(x=X_train, y=Y_train, batch_size=25, epochs=50)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 808       
                                                                 
 dense_1 (Dense)             (None, 8)                 72        
                                                                 
 dense_2 (Dense)             (None, 8)                 72        
                                                                 
 dropout (Dropout)           (None, 8)                 0         
                                                                 
 dense_3 (Dense)             (None, 1)                 9         
                                                                 
Total params: 961
Trainable params: 961
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
1400/1400 [==============================] - 2s

In [9]:
Y_test2 = Y_test[:, 0, :]
Y_pred = model.predict(X_test)
rms = np.sqrt(mean_squared_error(Y_test2, Y_pred))
print('Root Mean Square Error: ', rms)

218/218 [==============================] - 0s 726us/step
Root Mean Square Error:  0.21475667954620567


In [12]:
model = Sequential()
model.add(SimpleRNN(128, return_sequences=True, activation='relu',input_shape=(X_train.shape[1],1)))
model.add(Flatten())
model.add(Dense(X_train.shape[1], activation='relu'))
model.compile(optimizer='adam', loss='mae', metrics=['mean_squared_error'])
model.fit(X_train, Y_train, epochs=20)

Epoch 1/20
1094/1094 [==============================] - 28s 25ms/step - loss: 1.2413 - mean_squared_error: 4.4058
Epoch 2/20
1094/1094 [==============================] - 29s 27ms/step - loss: 1.1107 - mean_squared_error: 3.9957
Epoch 3/20
1094/1094 [==============================] - 27s 25ms/step - loss: 1.0294 - mean_squared_error: 3.7293
Epoch 4/20
1094/1094 [==============================] - 27s 24ms/step - loss: 0.9304 - mean_squared_error: 3.3731
Epoch 5/20
1094/1094 [==============================] - 27s 24ms/step - loss: 0.8754 - mean_squared_error: 3.1714
Epoch 6/20
1094/1094 [==============================] - 27s 24ms/step - loss: 0.8524 - mean_squared_error: 3.0880
Epoch 7/20
1094/1094 [==============================] - 27s 25ms/step - loss: 0.8175 - mean_squared_error: 2.9570
Epoch 8/20
1094/1094 [==============================] - 27s 25ms/step - loss: 0.7923 - mean_squared_error: 2.8655
Epoch 9/20
1094/1094 [==============================] - 27s 25ms/step - loss: 0.7754 - m

In [13]:
regress = Sequential()
regress.add(LSTM(units=200, return_sequences=True, input_shape=(X_train.shape[1], 1)))
regress.add(Dropout(0.3))
regress.add(LSTM(units=200, return_sequences=True))
regress.add(Dropout(0.3))
regress.add(LSTM(units=200, return_sequences=True))
regress.add(Dropout(0.3))
regress.add(LSTM(units=200))
regress.add(Dropout(0.3))
regress.add(Dense(units=1))
regress.compile(optimizer='adam',loss='mean_squared_error')
regress.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 100, 200)          161600    
                                                                 
 dropout_1 (Dropout)         (None, 100, 200)          0         
                                                                 
 lstm_2 (LSTM)               (None, 100, 200)          320800    
                                                                 
 dropout_2 (Dropout)         (None, 100, 200)          0         
                                                                 
 lstm_3 (LSTM)               (None, 100, 200)          320800    
                                                                 
 dropout_3 (Dropout)         (None, 100, 200)          0         
                                                                 
 lstm_4 (LSTM)               (None, 200)              

In [14]:
regress.fit(X_train,Y_train,epochs=100,batch_size=25)

Epoch 1/100
  77/1400 [>.............................] - ETA: 11:34 - loss: 0.5319

KeyboardInterrupt: ignored